In [1]:
# Data: http://saludata.saludcapital.gov.co/osb/datos_abiertos_osb/enf-transmisibles/OSB_EnfTransm-Uso-UCIS-COVID.csv

In [2]:
import requests
import pandas as pd
import re
import numpy

In [3]:
URL = 'http://saludata.saludcapital.gov.co/osb/datos_abiertos_osb/enf-transmisibles/OSB_EnfTransm-Uso-UCIS-COVID.csv'
FILEPATH = './data/uci_bogota_raw.csv'

In [4]:
r = requests.get( URL, stream = True )

with open( FILEPATH, 'wb' ) as f:
    f.write( r.content )

In [5]:
ucis = pd.read_csv( './data/uci_bogota_raw.csv', encoding = 'ISO-8859-1', sep = ';', skiprows = 4, skipfooter = 3 )

/Users/fabian/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [ ]:
def convert_int64(column:str):
    temp=ucis[column]
    i=0
    for row in temp:
        data=str(row) 
        search=re.findall('^[1-9]\.\w*',data)
        if(len(search)==1):
            #Select first item in list of re
            str1=search[0]
            
            #To float to aproximate 3 digits
            str1=float(str1)
            str1=round(str1,3)
            #Cast as str and replace '.'
            str1=str(str1).replace('.','')
            if(len(str1)==3):
                str1=str1+'0'
                
            #Replace in the column
            temp[i]=str1
            temp[i]=temp[i].astype(numpy.int64)
            
        else:
            row=float(data)
            temp[i]=int(row)
            temp[i]=temp[i].astype(numpy.int64)
        i+=1
    
    ucis[column]=temp.astype(numpy.int64)

convert_int64('Camas UCI ocupadas Covid-19')
convert_int64('Camas UCI Disponibles COVID 19')

In [6]:
ucis.shape

(98, 4)

In [7]:
ucis.dtypes

Fecha                             object
Camas UCI ocupadas Covid-19        int64
Camas UCI Disponibles COVID 19     int64
Ocupación UCI COVID 19            object
dtype: object

In [8]:
ucis[ 'Fecha' ] = pd.to_datetime( ucis[ 'Fecha' ], format = '%d/%m/%Y' )

In [9]:
ucis[ '% Ocupación' ] = ucis[ 'Camas UCI ocupadas Covid-19' ] / ucis[ 'Camas UCI Disponibles COVID 19' ]

In [10]:
ucis.dtypes

Fecha                             datetime64[ns]
Camas UCI ocupadas Covid-19                int64
Camas UCI Disponibles COVID 19             int64
Ocupación UCI COVID 19                    object
% Ocupación                              float64
dtype: object

In [11]:
del ucis[ 'Ocupación UCI COVID 19' ]

In [12]:
ucis.rename( columns = { '% Ocupación': 'Ocupacion' }, inplace = True )

In [13]:
ucis.tail( 20 )

,Fecha,Camas UCI ocupadas Covid-19,Camas UCI Disponibles COVID 19,Ocupacion
78,2020-06-26,617,883,0.698754
79,2020-06-27,637,911,0.699232
80,2020-06-28,670,939,0.713525
81,2020-06-29,691,942,0.733546
82,2020-06-30,694,957,0.725183
83,2020-07-01,729,988,0.737854
84,2020-07-02,748,966,0.774327
85,2020-07-03,792,993,0.797583
86,2020-07-04,811,998,0.812625
87,2020-07-05,849,1013,0.838105


In [14]:
ucis.to_csv( './data/uci_bogota.csv', index = False )